# Comprendre et manipuler la tokenisation dans les LLM

Ce notebook a pour objectif d’introduire la tokenisation, une étape fondamentale
dans le fonctionnement des modèles de langage de grande taille (LLM).

La tokenisation constitue l’interface entre le langage naturel et les modèles
numériques : elle détermine comment un texte est découpé, représenté et interprété
par un modèle.

À la fin de ce notebook, vous serez capable de :
- expliquer ce qu’est un token et un tokenizer,
- comparer différentes stratégies de tokenisation,
- manipuler des tokenizers modernes utilisés dans les LLM,
- comprendre l’impact de la tokenisation sur les performances et les coûts.


____

# Pourquoi la tokenisation est nécessaire
## 1. Du texte au numérique

Un modèle de langage est un modèle mathématique : il ne peut pas traiter directement
du texte brut. Toute information doit être représentée sous forme numérique.

La tokenisation est le processus qui permet de transformer un texte en une séquence
d’unités discrètes appelées *tokens*, lesquelles seront ensuite converties en
représentations numériques exploitables par le modèle.


## 2. Qu’est-ce qu’un token ?

Un token est l’unité de base manipulée par un modèle de langage.
Il ne correspond pas nécessairement à un mot entier : il peut s’agir d’une partie de
mot, d’un caractère, d’un nombre ou d’un symbole.

Le découpage en tokens dépend entièrement du *tokenizer* utilisé.
Un même texte peut donc être découpé différemment selon le modèle.


## 3. Qu’est-ce qu’un tokenizer ?

Un tokenizer est le composant chargé de transformer un texte brut en une séquence de tokens selon un ensemble de règles et de paramètres appris ou définis. 

Il détermine la manière dont le texte est découpé, quels symboles sont reconnus comme des unités valides et comment ces unités sont associées à des identifiants numériques. 

Le tokenizer joue un rôle central dans le fonctionnement d’un modèle de langage, car il conditionne la représentation du texte, la taille du vocabulaire, la longueur des séquences et, par conséquent, les performances et le coût computationnel du modèle. 

Chaque tokenizer est spécifiquement conçu pour être compatible avec un modèle donné, et toute incohérence entre le tokenizer et le modèle entraîne une dégradation significative des résultats.

____

# Les différentes tokenisations

## Tokenisation naïve par mots

Le texte est découpé en mots entiers séparés par des espaces ou des signes de ponctuation : 

In [1]:
text = "Les modèles de langage sont fascinants."
tokens = text.split()
tokens

['Les', 'modèles', 'de', 'langage', 'sont', 'fascinants.']

Cette approche est simple mais pose rapidement problème :
- le vocabulaire devient très grand,
- les mots inconnus ne peuvent pas être représentés,
- la généralisation est limitée.

## Tokenisation par caractères

Chaque caractère du texte est traité comme un token, garantissant une couverture complète du vocabulaire au prix de séquences très longues : 

In [3]:
list(text)

['L',
 'e',
 's',
 ' ',
 'm',
 'o',
 'd',
 'è',
 'l',
 'e',
 's',
 ' ',
 'd',
 'e',
 ' ',
 'l',
 'a',
 'n',
 'g',
 'a',
 'g',
 'e',
 ' ',
 's',
 'o',
 'n',
 't',
 ' ',
 'f',
 'a',
 's',
 'c',
 'i',
 'n',
 'a',
 'n',
 't',
 's',
 '.']

La tokenisation par caractères résout le problème des mots inconnus,
mais elle produit des séquences très longues, ce qui augmente fortement
le coût computationnel.

## Tokenisation par sous-mots

Les LLM modernes utilisent majoritairement des tokenizations basées sur des sous-mots.
L’idée est de découper les mots en unités fréquentes, réutilisables, permettant
de gérer efficacement les mots rares ou inconnus.

Exemple :
- "incompréhensible" → "in" + "compréhens" + "ible"


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
text = "Tokenization is a crucial step in natural language processing."
tokens = tokenizer.tokenize(text)
tokens


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

['token',
 '##ization',
 'is',
 'a',
 'crucial',
 'step',
 'in',
 'natural',
 'language',
 'processing',
 '.']

On observe que le texte est découpé en sous-unités, et non en mots stricts.


In [5]:
token_ids = tokenizer.encode(text)
token_ids

[101, 19204, 3989, 2003, 1037, 10232, 3357, 1999, 3019, 2653, 6364, 1012, 102]

Les modèles ne manipulent pas directement les tokens textuels, mais leurs identifiants numériques associés.


## Tokenisation au niveau des bytes (byte-level tokenization)  

Le texte est tokenisé à partir des octets qui le composent, assurant une représentation universelle et indépendante de la langue.

In [10]:
text = "Prix : 199,99€ 😄"

# Encodage UTF-8 en bytes
byte_tokens = text.encode("utf-8")

byte_tokens


b'Prix : 199,99\xe2\x82\xac \xf0\x9f\x98\x84'

In [11]:
list(byte_tokens)

[80,
 114,
 105,
 120,
 32,
 58,
 32,
 49,
 57,
 57,
 44,
 57,
 57,
 226,
 130,
 172,
 32,
 240,
 159,
 152,
 132]

In [12]:
bytes(byte_tokens).decode("utf-8")

'Prix : 199,99€ 😄'

## Tokenisation hybride 

Plusieurs niveaux de tokenisation sont combinés afin d’équilibrer robustesse, efficacité et précision selon le contexte d’utilisation.

Principe :
* on tokenise d’abord par mots connus,
* sinon on découpe en sous-mots,
* sinon on retombe au niveau byte ou caractère.

In [13]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

text = "ChatGPTification ultra-méta-quantique 😄"
tokens = tokenizer.tokenize(text)

tokens

['Chat',
 'G',
 'PT',
 'ification',
 'Ġultra',
 '-',
 'm',
 'Ã©',
 'ta',
 '-',
 'quant',
 'ique',
 'ĠðŁĺ',
 'Ħ']

In [14]:
tokenizer.encode(text)

[30820,
 38,
 11571,
 2649,
 14764,
 12,
 76,
 2634,
 8326,
 12,
 40972,
 2350,
 30325,
 226]

## Reconstruction du texte

In [6]:
tokenizer.convert_ids_to_tokens(token_ids)

['[CLS]',
 'token',
 '##ization',
 'is',
 'a',
 'crucial',
 'step',
 'in',
 'natural',
 'language',
 'processing',
 '.',
 '[SEP]']

In [7]:
tokenizer.decode(token_ids)

'[CLS] tokenization is a crucial step in natural language processing. [SEP]'

## Comparaison de tokenizers

In [8]:
models = [
    "bert-base-uncased",
    "gpt2",
    "xlm-roberta-base"
]

text = "L'internationalisation des modèles de langage est complexe."

for model in models:
    tok = AutoTokenizer.from_pretrained(model)
    print(f"\nTokenizer: {model}")
    print(tok.tokenize(text))
    print("Nombre de tokens:", len(tok.tokenize(text)))



Tokenizer: bert-base-uncased
['l', "'", 'international', '##isation', 'des', 'model', '##es', 'de', 'lang', '##age', 'est', 'complex', '##e', '.']
Nombre de tokens: 14


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]


Tokenizer: xlm-roberta-base
['▁L', "'", 'international', 'isation', '▁des', '▁modèles', '▁de', '▁lang', 'age', '▁est', '▁complexe', '.']
Nombre de tokens: 12


## Impact pratique de la tokenisation

Les limites de contexte des LLM sont exprimées en nombre de tokens,
pas en nombre de mots ou de caractères.

Deux textes de longueur similaire peuvent donc avoir des coûts très différents
selon leur tokenisation.


## Cas limites

In [9]:
texts = [
    "ChatGPTification ultra-méta-quantique",
    "Prix : 199,99€ 😄",
    "anticonstitutionnellement"
]

for t in texts:
    print(t)
    print(tokenizer.tokenize(t))
    print()


ChatGPTification ultra-méta-quantique
['chat', '##gp', '##ti', '##fication', 'ultra', '-', 'meta', '-', 'quan', '##tique']

Prix : 199,99€ 😄
['prix', ':', '199', ',', '99', '##€', '[UNK]']

anticonstitutionnellement
['anti', '##con', '##sti', '##tu', '##tion', '##nell', '##ement']



## Conclusion

La tokenisation est une étape fondamentale dans les LLM.
Elle conditionne la représentation du texte, la longueur des séquences,
les performances des modèles et leur coût d’utilisation.

Comprendre la tokenisation, c’est comprendre comment un modèle de langage
lit réellement le langage humain.
